In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Опис даних:

squareMeters - житлова площа (м2) </br>
numberOfRooms - кількість кімнат </br>
hasYard </br>
hasPool </br>
floors - number of floors </br>
cityCode - zip code </br>
cityPartRange - the higher the range, the more exclusive the neighbourhood is </br>
numPrevOwners - number of prevoious owners </br>
made - year </br>
isNewBuilt </br>
hasStormProtector </br>
basement - basement square meters </br>
attic - attic square meteres </br>
garage - garage size </br>
hasStorageRoom </br>
hasGuestRoom - number of guest rooms </br>
price - predicted value </br>

In [4]:
data = pd.read_csv("train.csv", index_col = 'id')
data.head()

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
id,,,,,,,,,,,,,,,,,
0,34291,24,1,0,47,35693,2,1,2000,0,1,8,5196,369,0,3,3436795.2
1,95145,60,0,1,60,34773,1,4,2000,0,1,729,4496,277,0,6,9519958.0
2,92661,45,1,1,62,45457,4,8,2020,1,1,7473,8953,245,1,9,9276448.1
3,97184,99,0,0,59,15113,1,1,2000,0,1,6424,8522,256,1,9,9725732.2
4,61752,100,0,0,57,64245,8,4,2018,1,0,7151,2786,863,0,7,6181908.8
